##Création et import des données dans SQLite

In [ ]:
#Import des données dasn SQLite :


CREATE TABLE REGION (
                code_region VARCHAR(3) NOT NULL,
                nom_region VARCHAR(100) NOT NULL,
                CONSTRAINT pk_region PRIMARY KEY (code_region)
);


CREATE TABLE DEPARTEMENT (
                code_departement VARCHAR(5) NOT NULL,
                code_region VARCHAR(3) NOT NULL,
                nom_departement VARCHAR(100) NOT NULL,
                CONSTRAINT pk_departement PRIMARY KEY (code_departement)
);


CREATE TABLE COMMUNE (
                id_codedep_codecommune VARCHAR(10) NOT NULL,
                code_departement VARCHAR(5) NOT NULL,
                code_commune VARCHAR(5) NOT NULL,
                nom_commune VARCHAR(50) NOT NULL,
                population INTEGER NOT NULL,
                CONSTRAINT pk_commune PRIMARY KEY (id_codedep_codecommune)
);


CREATE TABLE BIEN (
                id_bien VARCHAR NOT NULL,
                id_codedep_codecommune VARCHAR(10) NOT NULL,
                no_voie VARCHAR(10) NOT NULL,
                type_voie VARCHAR(10),
                voie VARCHAR(50) NOT NULL,
                total_piece INTEGER NOT NULL,
                surface_carrez FLOAT NOT NULL,
                surface_reel_bati INTEGER NOT NULL,
                type_local VARCHAR(50) NOT NULL,
                BTQ VARCHAR(3),
                CONSTRAINT pk_bien PRIMARY KEY (id_bien)
);


CREATE TABLE VENTE (
                id_vente VARCHAR NOT NULL,
                id_bien VARCHAR NOT NULL,
                date_vente DATE NOT NULL,
                valeur FLOAT,
                CONSTRAINT pk_vente PRIMARY KEY (id_vente)
);


##Requete 1 : Nombre total d’appartements vendus au 1er semestre 2020

In [ ]:
SELECT COUNT(b.id_bien) AS nombre_appartement_vendu_S1_2020  # Agrégation : on compte les biens
FROM BIEN b                                                  # Depuis la table BIEN
INNER JOIN VENTE v ON b.id_bien = v.id_bien                  # Jointure interne : ne garde que les biens qui ont une vente correspondante
WHERE b.type_local = 'Appartement'                           # Filtre pour ne garder que les appartements
AND v.date_vente                                             # Filtre sur la date pour conserver la période concernée
BETWEEN '2020-01-01' AND '2020-06-30'

LEFT JOIN + filtre dans le WHERE => même résultat qu’un INNER JOIN, donc pas d’intérêt particulier ici.

LEFT JOIN + filtre dans le ON => permettrait aussi de répertorier les appartements sans vente, mais ce n’est pas la question posée.

NATURAL JOIN => se comporte comme un INNER JOIN mais reste fragile et moins lisible ; pas recommandé en production.

##Requete 2 : Le nombre de ventes d’appartement par région pour le 1er semestre 2020

In [ ]:
SELECT
  r.nom_region AS région,                                   # Sélectionne chaque région et le nombre de ventes
  COUNT(*) AS nb_ventes_appartement_S1_2020                 # Agrégation : nombre de ventes
FROM VENTE v                                                # Dans la table vente
  INNER JOIN BIEN b                                         # Jointure avec BIEN pour filtrer par type_local
    ON v.id_bien = b.id_bien
  INNER JOIN COMMUNE c                                      # Jointure géographique : BIEN → COMMUNE
    ON b.id_codedep_codecommune = c.id_codedep_codecommune
  INNER JOIN DEPARTEMENT d                                  # Jointure géographique : COMMUNE → DÉPARTEMENT
    ON c.code_departement = d.code_departement
  INNER JOIN REGION r                                       # Jointure géographique : DÉPARTEMENT → RÉGION
    ON d.code_region = r.code_region
WHERE b.type_local = 'Appartement'                          # Filtre : appartements uniquement
  AND v.date_vente
  BETWEEN '2020-01-01' AND '2020-06-30'                     # Filtre : ventes 1er semestre 2020
GROUP BY
  r.nom_region                                              # Agrège par région
ORDER BY
  nb_ventes_appartement_S1_2020 DESC                        # Classe les régions du plus grand au plus petit nombre

##Requete 3 : Proportion des ventes d’appartements par le nombre de pièces

In [ ]:
SELECT                              # Sélection des colonnes souhaitées
  b.total_piece AS pieces,          # Nombre de pièces de chaque appartement
  COUNT(*)      AS nb_ventes,       # Nombre de ventes pour ce nombre de pièces
  ROUND(                            # Arrondi du calcul du pourcentage
    100.0 * COUNT(*)                # Multiplication par 100 pour obtenir un pourcentage
    / (
        SELECT COUNT(*)              # Sous-requête : nombre total de ventes d’appartements
        FROM VENTE v                 # Table des ventes (alias v)
        INNER JOIN BIEN b            # Jointure interne avec la table des biens (alias b)
          ON v.id_bien = b.id_bien
        WHERE b.type_local = 'Appartement'  # Filtre pour ne garder que les appartements
      ), 2                             # Deuxième argument : arrondir à 2 décimales
  ) AS pct_ventes                     # Alias pour la colonne du pourcentage
FROM VENTE v
INNER JOIN BIEN b                     # Jointure interne avec la table des biens
  ON v.id_bien = b.id_bien
WHERE b.type_local = 'Appartement'    # Filtre pour ne sélectionner que les appartements
GROUP BY b.total_piece                # Regroupement des résultats par nombre de pièces
ORDER BY b.total_piece;               # Tri des résultats par nombre de pièces croissant


##Requete 4 : Liste des 10 départements où le prix du mètre carré est le plus élevé

In [ ]:
SELECT
  d.nom_departement AS departement,                       # Libellé du département pour l’affichage
  ROUND(AVG(v.valeur / b.surface_carrez), 0) AS prix_m2   # Prix moyen au m², arrondi à l’euro
FROM VENTE v
INNER JOIN BIEN b                                         # Jointure avec BIEN pour la surface
  ON v.id_bien = b.id_bien
INNER JOIN COMMUNE c                                      # Jointure géographique : bien → commune
  ON b.id_codedep_codecommune = c.id_codedep_codecommune
INNER JOIN DEPARTEMENT d                                  # Jointure géographique : commune → département
  ON c.code_departement = d.code_departement
WHERE b.type_local = 'Appartement'                        # Filtre : ne garder que les appartements
  AND b.surface_carrez > 0                                # Filtre : surface positive (évite division par 0)
GROUP BY
  c.code_departement, d.nom_departement                   # Agrégation par département
ORDER BY
  prix_m2 DESC                                            # Classement décroissant (du plus cher au moins cher)
LIMIT 10;                                                 # Ne conserver que les 10 départements les plus chers


##Requete 5 : Prix moyen du mètre carré d’une maison en Île-de-France

In [ ]:
SELECT
  ROUND(AVG(v.valeur / b.surface_carrez), 0) AS prix_moyen_maison_IDF   # Prix moyen €/m², arrondi à l’euro
FROM VENTE v
INNER JOIN BIEN b                                                       # Jointure vente → bien pour récupérer la surface
  ON v.id_bien = b.id_bien
INNER JOIN COMMUNE c                                                    # Jointure géographique : bien → commune
  ON b.id_codedep_codecommune = c.id_codedep_codecommune
INNER JOIN DEPARTEMENT d                                                # Jointure géographique : commune → département
  ON c.code_departement = d.code_departement
INNER JOIN REGION r                                                     # Jointure géographique : département → région
  ON d.code_region = r.code_region
WHERE UPPER(TRIM(r.nom_region)) = 'ILE-DE-FRANCE'                       # Filtre : région Île-de-France (normalisée en majuscules)
  AND b.type_local = 'Maison'                                           # Filtre : ne garder que les maisons
  AND b.surface_carrez > 0;                                             # Filtre : surface positive (évite division par 0)


##Requete 6 : Liste des 10 appartements les plus chers avec la région et le nombre de mètres carrés

In [ ]:
# Sélection des colonnes à afficher
SELECT
  v.id_bien AS id_bien,        # Identifiant unique du bien vendu
  v.valeur  AS Prix,           # Prix de vente du bien
  b.surface_carrez AS Surface,        # Surface Carrez du bien (m²)
  r.nom_region     AS Region          # Nom de la région du bien

# Source principale : table des ventes
FROM VENTE v                                 # Table des ventes (alias v)

# Jointures pour enrichir les données
INNER JOIN BIEN b                           # Lie chaque vente à son bien
  ON v.id_bien = b.id_bien                  # Condition de jointure VENTE ↔ BIEN

INNER JOIN COMMUNE c                        # Lie le bien à sa commune
  ON b.id_codedep_codecommune = c.id_codedep_codecommune

INNER JOIN DEPARTEMENT d                    # Lie la commune à son département
  ON c.code_departement = d.code_departement

INNER JOIN REGION r                         # Lie le département à sa région
  ON d.code_region = r.code_region


WHERE b.type_local = 'Appartement'          # Frilte uniquement les appartements
  AND v.valeur <> ''                        # Exclure les enregistrements sans prix renseigné

ORDER BY v.valeur DESC                      # Trier du prix le plus élevé au plus faible
LIMIT 10;                                   # Afficher les 10 premiers résultats


##Requete 7 : Taux d’évolution du nombre de ventes entre le premier et le second trimestre de 2020

In [ ]:
WITH
  ventes_t1 AS (                              # CTE 1 : total des ventes au 1ᵉʳ trimestre 2020
    SELECT COUNT(*) AS nb_t1                  #  ↳ calcule nb_t1
    FROM VENTE                                #      dans la table VENTE
    WHERE date_vente BETWEEN '2020-01-01'     #      du 1 janv.
                        AND '2020-03-31'      #      au 31 mars 2020 inclus
  ),

  ventes_t2 AS (                              # CTE 2 : total des ventes au 2ᵉ trimestre 2020
    SELECT COUNT(*) AS nb_t2                  #  ↳ calcule nb_t2
    FROM VENTE
    WHERE date_vente BETWEEN '2020-04-01'     #      du 1 avr.
                        AND '2020-06-30'      #      au 30 juin 2020 inclus
  )

SELECT                                        # Requête finale : taux d’évolution T2 vs T1
  ROUND(
    (ventes_t2.nb_t2 - ventes_t1.nb_t1)       #   différence absolue de volume
    * 100.0 / ventes_t1.nb_t1,                #   convertie en pourcentage
    2                                         #   arrondie à 2 décimales
  ) AS taux_evolution                         # Alias de la colonne résultat
FROM ventes_t1, ventes_t2;                    # Produit cartésien des deux CTE (1 seule ligne)


##Requete 8 : Le classement des régions par rapport au prix au mètre carré desappartement de plus de 4 pièces

In [ ]:
SELECT
  r.nom_region AS Region,                                  # Nom de la région
  ROUND(AVG(v.valeur / b.surface_carrez),                  # Moyenne des ratios valeur/surface
    2                                                      # Deux décimales
  ) AS prix_m2                                             # Prix moyen au m² arrondi

FROM VENTE v
INNER JOIN BIEN b                                         # Lier chaque vente à son bien
  ON v.id_bien = b.id_bien
INNER JOIN COMMUNE c                                      # Lier le bien à sa commune
  ON b.id_codedep_codecommune = c.id_codedep_codecommune
INNER JOIN DEPARTEMENT d                                  # Lier la commune à son département
  ON c.code_departement = d.code_departement
INNER JOIN REGION r                                       # Lier le département à sa région
  ON d.code_region = r.code_region

WHERE
  b.type_local    = 'Appartement'    # Filtrer sur les appartements
  AND b.total_piece > 4              # Plus de 4 pièces
  AND v.valeur     IS NOT NULL       # Prix renseigné
  AND b.surface_carrez > 0           # Surface valide non nulle

GROUP BY
  r.nom_region                      # Regrouper par région

ORDER BY
  prix_m2 DESC;                     # Classement du plus cher au moins cher


##Requete 9 : Liste des communes ayant eu au moins 50 ventes au 1er trimestre

In [ ]:
WITH ventes_t1 AS (                               # CTE : compte les ventes par commune
  SELECT
    b.id_codedep_codecommune AS code_insee,       # Identifiant INSEE (clé unique pour distinguer les homonymes)
    c.nom_commune            AS commune,          # Libellé de la commune
    COUNT(*)                 AS nb_ventes         # Nombre de ventes enregistrées sur la période
  FROM VENTE v                                    # Table des transactions
  JOIN BIEN b                                     # Relie chaque vente à son bien
    ON v.id_bien = b.id_bien
  JOIN COMMUNE c                                  # Relie le bien à sa commune
    ON b.id_codedep_codecommune = c.id_codedep_codecommune
  WHERE v.date_vente BETWEEN '2020-01-01'         # Début T1 2020
                       AND '2020-03-31'           # Fin   T1 2020
  GROUP BY
    b.id_codedep_codecommune,                     # Grain = commune unique
    c.nom_commune
)

SELECT
  commune,                                        # Affiche uniquement le libellé…
  nb_ventes                                       # …et le volume de ventes
FROM ventes_t1
WHERE nb_ventes >= 50                             # Seuil d’activité à 50 ventes
ORDER BY nb_ventes DESC;                          # Classement décroissant


##Requete 10 : Différence en pourcentage du prix au mètre carré entre un appartement de 2 pièces et un appartement de 3 pièces

In [ ]:
# T3 = 4135 euro du m2 et T2 = 4727 euro du m2.

#Écart du T3 par rapport au T2  (référence = T2)
((4135 - 4727) / 4727) * 100   =  -12.523799449968268 %

# Écart du T2 par rapport au T3  (référence = T3)
((4727 - 4135) / 4135) * 100   =   14.316807738814993 %


WITH stats AS (
  SELECT
    b.total_piece                       AS pieces,        # Nombre de pièces (2 ou 3)
    COUNT(*)                            AS nb_ventes,     # Nombre de ventes pour ce type
    AVG(v.valeur / b.surface_carrez)    AS prix_m2        # Prix moyen simple (€ / m²)
  FROM VENTE v
  INNER JOIN BIEN b
    ON v.id_bien = b.id_bien            # Jointure ventes ↔ biens
  WHERE
    b.type_local    = 'Appartement'     # Filtrer sur les appartements
    AND v.valeur     IS NOT NULL        # Prix renseigné
    AND b.surface_carrez > 0            # Surface valide
    AND b.total_piece IN (2, 3)         # Seulement 2 et 3 pièces
  GROUP BY
    b.total_piece                       # Regrouper par nombre de pièces
)
SELECT
  s2.nb_ventes      AS nb_ventes_2,     # Nombre de ventes pour 2-pièces
  s2.prix_m2        AS prix_m2_2,       # Prix moyen au m² pour 2-pièces
  s3.nb_ventes      AS nb_ventes_3,     # Nombre de ventes pour 3-pièces
  s3.prix_m2        AS prix_m2_3,       # Prix moyen au m² pour 3-pièces
  ROUND(
    100.0 * (s3.prix_m2 - s2.prix_m2)    # (prix3 – prix2) × 100 / prix2
    / s2.prix_m2
  , 2)               AS difference_pct   # Écart en % arrondi à 2 décimales
FROM stats s2
INNER JOIN stats s3
  ON s2.pieces = 2                       # S2 = stats pour 2-pièces
  AND s3.pieces = 3;                     # S3 = stats pour 3-pièces


##Requete 11 : Les moyennes de valeurs foncières pour le top 3 des communes des départements 6, 13, 33, 59 et 69

In [ ]:
WITH sales_commune AS (                         # CTE 1 : agrège les ventes par commune
  SELECT
    c.code_departement,                         # Code INSEE du département
    c.nom_commune,                              # Nom de la commune
    COUNT(*)      AS nb_ventes,                 # Nombre total de ventes dans la commune
    AVG(v.valeur) AS avg_valeur                 # Valeur foncière moyenne (€)
  FROM VENTE v
  INNER JOIN BIEN b                             # Relie chaque vente à son bien
    ON v.id_bien = b.id_bien                    #  ↳ clé de jointure : id_bien
  INNER JOIN COMMUNE c                          # Relie le bien à sa commune
    ON b.id_codedep_codecommune = c.id_codedep_codecommune
  WHERE c.code_departement IN ('06','13','33','59','69')  # Filtre : 5 départements ciblés
    AND v.valeur IS NOT NULL                   # Exclut les ventes sans prix
  GROUP BY
    c.code_departement,                        # Regroupement par département
    c.nom_commune                              # … puis par commune
),                                              # Fin CTE 1

ranked_communes AS (                            # CTE 2 : attribue un rang par département
  SELECT
    code_departement,
    nom_commune,
    nb_ventes,
    avg_valeur,
    RANK() OVER (                               # Fonction analytique : calcule un rang
      PARTITION BY code_departement             # → redémarre le classement pour chaque département
      ORDER BY nb_ventes DESC                   # → rang 1 = commune la plus vendue
    ) AS rk
  FROM sales_commune                            # Source : résultats du CTE 1
)

SELECT                                          # Requête finale : extrait le Top 3
  code_departement,                             # Département
  nom_commune AS commune,                       # Nom de la commune
  ROUND(avg_valeur, 2) AS moyenne_valeur_fonciere  # Prix moyen arrondi à 2 décimales
FROM ranked_communes
WHERE rk <= 3                                   # Ne conserver que les rangs 1, 2, 3
ORDER BY
  code_departement,                             # Tri d’abord par département
  rk;                                           # Puis par rang croissant




##AVEC ROW NUMBER

WITH sales_commune AS (
  SELECT
    c.code_departement,
    c.nom_commune,
    COUNT(*)      AS nb_ventes,
    AVG(v.valeur) AS avg_valeur
  FROM VENTE v
  INNER JOIN BIEN b
    ON v.id_bien = b.id_bien
  INNER JOIN COMMUNE c
    ON b.id_codedep_codecommune = c.id_codedep_codecommune
  WHERE c.code_departement IN ('06','13','33','59','69')
    AND v.valeur IS NOT NULL
  GROUP BY
    c.code_departement,
    c.nom_commune
),

ranked_communes AS (
  SELECT
    code_departement,
    nom_commune,
    nb_ventes,
    avg_valeur,
    ROW_NUMBER() OVER (
      PARTITION BY code_departement
      ORDER BY nb_ventes DESC
    ) AS rn
  FROM sales_commune
)

SELECT
  code_departement,
  nom_commune AS commune,
  ROUND(avg_valeur, 2) AS moyenne_valeur_fonciere
FROM ranked_communes
WHERE rn <= 3
ORDER BY
  code_departement,
  rn;


##Requete 12 : Les 20 communes avec le plus de transactions pour 1000 habitants pour les communes qui dépassent les 10 000 habitants


In [ ]:
WITH tx_per_commune AS (                                   # CTE : calcule le volume par commune
  SELECT
    c.nom_commune             AS commune,                  # Nom de la commune
    c.population,                                         # Population municipale
    COUNT(*)                  AS nb_transactions           # Nombre total de ventes réalisées
  FROM VENTE v                                             # Table des transactions
  INNER JOIN BIEN b
    ON v.id_bien = b.id_bien                               # Relie chaque vente à son bien
  INNER JOIN COMMUNE c
    ON b.id_codedep_codecommune = c.id_codedep_codecommune # Relie le bien à sa commune
  WHERE
    v.valeur IS NOT NULL                                   # Exclut les ventes sans prix renseigné
    AND c.population > 10000                               # Ne retient que les communes de >10 000 habitants
  GROUP BY
    c.nom_commune,                                         # Regroupe par commune…
    c.population                                           # …et conserve la population pour le calcul du taux
)

SELECT
  commune,                                                 # Nom de la commune
  nb_transactions,                                         # Volume total de ventes
  population,                                              # Population municipale
  ROUND(nb_transactions * 1000.0 / population, 2) AS tx_transactions_pour_1000
                                                           # Taux de transactions pour 1 000 habitants
FROM tx_per_commune                                        # Source : résultats agrégés du CTE
ORDER BY
  tx_transactions_pour_1000 DESC                           # Classement décroissant sur le taux
LIMIT 20;                                                  # Affiche uniquement les 20 meilleures communes
